<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import category_encoders as ce
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# Load the data
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn (1).csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [4]:
# Split the data
X = df.drop(columns=['Churn', 'customerID'])
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5634, 19), (5634,), (1409, 19), (1409,))

In [5]:
def clean(X_train, X_test):
    """Function to scale and encode the X variables"""
    encoder = ce.OrdinalEncoder()
    scaler = StandardScaler()
    X_train = X_train.copy()
    X_test = X_test.copy()
    X_train = encoder.fit_transform(X_train)
    X_test = encoder.transform(X_test)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test

In [6]:
X_train, X_test = clean(X_train, X_test)

In [7]:
# Convert train and test targets to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [8]:
# Train a baseline model
# Parameters
inputs = X_train.shape[1]
epochs = 50
batch_size = 10

# Create Model
model = Sequential()
model.add(Dense(15, activation='relu', input_shape=(inputs,)))
model.add(Dense(15, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Fit Model
model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=epochs,
          batch_size=batch_size,
          verbose=True
         )

Train on 5634 samples, validate on 1409 samples
Epoch 1/50
5634/5634 [==============================] - 3s 489us/sample - loss: 0.4950 - acc: 0.7426 - val_loss: 0.4272 - val_acc: 0.7913
Epoch 2/50
5634/5634 [==============================] - 2s 318us/sample - loss: 0.4395 - acc: 0.7870 - val_loss: 0.4182 - val_acc: 0.8055
Epoch 3/50
5634/5634 [==============================] - 2s 319us/sample - loss: 0.4308 - acc: 0.7925 - val_loss: 0.4154 - val_acc: 0.8013
Epoch 4/50
5634/5634 [==============================] - 2s 318us/sample - loss: 0.4260 - acc: 0.7961 - val_loss: 0.4137 - val_acc: 0.8062
Epoch 5/50
5634/5634 [==============================] - 2s 308us/sample - loss: 0.4231 - acc: 0.7975 - val_loss: 0.4114 - val_acc: 0.8119
Epoch 6/50
5634/5634 [==============================] - 2s 309us/sample - loss: 0.4208 - acc: 0.8028 - val_loss: 0.4108 - val_acc: 0.8070
Epoch 7/50
5634/5634 [==============================] - 2s 320us/sample - loss: 0.4183 - acc: 0.8010 - val_loss: 0.4162 - va

In [11]:
import wandb
# from wandb.tensorflow import WandbHook
# wandb.init(project="customer-churn-ds6", entity="theta_158")

# Set random seed for reproducibility
seed = 7
np.random.seed(seed)

# Function to create model
def create_model():
    # Create model
    model = Sequential()
    model.add(Dense(15, input_dim=19, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define grid search parameters
params = {'batch_size': [28, 30, 32],
          'epochs': [48, 50, 52]}

# Create randomized search
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy', n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7933972310969116 using {'batch_size': 32, 'epochs': 50}
Means: 0.7825701100461484, Stdev: 0.007257895995146729 with: {'batch_size': 28, 'epochs': 48}
Means: 0.7882499112531062, Stdev: 0.01024027483489837 with: {'batch_size': 28, 'epochs': 50}
Means: 0.7891373801916933, Stdev: 0.011135983532381403 with: {'batch_size': 28, 'epochs': 52}
Means: 0.7912673056443025, Stdev: 0.010589443757261011 with: {'batch_size': 30, 'epochs': 48}
Means: 0.7882499112531062, Stdev: 0.0067830519861546415 with: {'batch_size': 30, 'epochs': 50}
Means: 0.7875399361022364, Stdev: 0.010721225402491813 with: {'batch_size': 30, 'epochs': 52}
Means: 0.7882499112531062, Stdev: 0.012689613341698344 with: {'batch_size': 32, 'epochs': 48}
Means: 0.7933972310969116, Stdev: 0.010569496267701347 with: {'batch_size': 32, 'epochs': 50}
Means: 0.7900248491302805, Stdev: 0.010062470690083371 with: {'batch_size': 32, 'epochs': 52}


In [12]:
from sklearn.metrics import accuracy_score

y_pred = grid_result.best_estimator_.predict(X_test)
score = accuracy_score(y_test, y_pred)
score

0.8097941802696949

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?